In [53]:
import requests
import json
import time
import logging

class WebSim(object):
    '''
    Example:
    ws = websim.WebSim(login = 'gosha-bor@yandex.ru', password = '...')
    ws.authorise()
    ws.send_alpha('-returns')
    '''
    def __init__(self, login = None, password = None, UID = None, WSSID = None, account_id = None):
        self.login = login
        self.password = password
        self.UID = UID
        self.WSSID = WSSID
        self.account_id = account_id
        self.headers = {
            'Host': 'websim.worldquantchallenge.com',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:52.0) Gecko/20100101 Firefox/52.0',
            'Accept': 'application/json',
            'Accept-Language':'en-US,en;q=0.5',
            'Cache-Control': 'max-age=0',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Origin': 'https://websim.worldquantchallenge.com',
            'Referer': 'https://websim.worldquantchallenge.com/result?q=9190149',
            'Connection': 'keep-alive',
        }

    def send_alpha(self, code, delay = "0", decay = 10, opneut = "subindustry", optrunc = "0.1", univid = "TOP3000", backdays = 256):
        cookies = {
            'django_language': 'en',
        #    'sessionid': 'p50gmiem9hykheowx9vqp0o2k15ls3gh',
        #    '_xsrf': '2|c1d41832|6d1377ea652ff73e906cd52084aa5255|1491915833',
        #    'UID': 'd6f1c3b48fe0f907e06f3e4555c2a5af1c23c56e4ee7f4a9e84f864d13d1955e',
        #    'WSSID':'"2|1:0|10:1507563160|5:WSSID|60:RlNXbzRKTDFIdEc2OStJMzExR05WVHBPK3p6UEI1Z0IxKzdGb3pCSjNnMD0=|7f9c94134a9b5b4852f082537e50c91faed0fea4a801491017006bbe01ac5098"',
            'UID': self.UID,
            'WSSID': self.WSSID,
        }
        
#        data = json.dump()
        
        r = requests.post('https://websim.worldquantchallenge.com/simulate',
            headers = self.headers,
            cookies = cookies,
            data = 'args=' + json.dumps([{
                'delay': delay,
                'unitcheck': 'off',
                'univid': univid,
                'opcodetype': 'EXPRESSION',
                'opassetclass': 'EQUITY',
                'optrunc': optrunc,
                'code': code,
                'region': 'USA',
                'opneut': opneut,
                #'IntradayType':'null',
                'tags': 'equity',
                'decay': decay,
                'DataViz': 0,
                'backdays': backdays,
                'simtime': 'Y5',
            }])
        )
#                 'IntradayType': 'null',       
#        print r
        print('\nsimulate...')
        print(r.text)
        job = json.loads(r.text)
        if not job['status']:
            raise ValueError(job)
        job_id = job['result'][0]

#        print 'sending all'
        r = requests.post('https://websim.worldquantchallenge.com/simulate/settings/all',
            headers = self.headers,
            cookies = cookies,
        )
        print('\nsettings/all:')
        print(r.text)
        all_ = json.loads(r.text)

#        print 'details'
        r = requests.post('https://websim.worldquantchallenge.com/job/details/{}'.format(job_id),
            headers = self.headers,
            cookies = cookies,
        )
        print('\ndetails:')
        print(r.text)
        details = json.loads(r.text)

        while True:
            time.sleep(3)

            print('\nrequest progress...')
            r = requests.post('https://websim.worldquantchallenge.com/job/progress/{}'.format(job_id),
                headers = self.headers,
                cookies = cookies,
            )
            print(r.text)
            
            if r.text == "\"DONE\"":
                break
            if r.text == "\"ERROR\"":
                #raise ValueError('Got ERROR in process request')
                return 'ERROR'
        

#        print 'details after done'
        r = requests.post('https://websim.worldquantchallenge.com/job/details/{}'.format(job_id),
            headers = self.headers,
            cookies = cookies,
        )
#        print r
#        print r.text
        details2 = json.loads(r.text)

        alphaID = details2['result']['clientAlphaId']
        

        r = requests.post('https://websim.worldquantchallenge.com/job/pnlchart/{}'.format(job_id),
            headers = self.headers,
            cookies = cookies,
        )
        pnl_chart = json.loads(r.text)

        r = requests.post('https://websim.worldquantchallenge.com/job/grade/{}'.format(job_id),
            headers = self.headers,
            cookies = cookies,
        )
        grade = json.loads(r.text)

        r = requests.post('https://websim.worldquantchallenge.com/job/simsummary/{}'.format(job_id),
            headers = self.headers,
            cookies = cookies,
        )
        simsummary = json.loads(r.text)

        r = requests.post('https://websim.worldquantchallenge.com/job/simsummary/{}'.format(job_id),
            headers = self.headers,
            cookies = cookies,
        )
        simsummary = json.loads(r.text)
        return {
            'alphaID': alphaID,
            'grade': grade,
            'pnl_chart': pnl_chart,
            'simsummary': simsummary,
        }

    def authorise(self):
        cookies = {
        }
        r = requests.post('https://websim.worldquantchallenge.com/login/process',
            headers = self.headers,
            cookies = cookies,
            data = {
                'EmailAddress': self.login,
                'Password': self.password,
                'next':'/dashboard',
                'g-recaptcha-response': '',
            }
        )
        print(r)
        print(r.text)
        #print(r.cookies)
        cookies_ = r.cookies.get_dict(domain = '.worldquantchallenge.com')
        #print r.cookies
        print("COOKIES:", cookies_)
        self.UID = cookies_['UID']
        self.WSSID = cookies_['WSSID']

In [54]:
ws = WebSim(login = 'dsd_kem@mail.ru', password = 'NBSn6ZNe#1')
ws.authorise()

<Response [200]>
{"status": true, "result": {"redirect": "/dashboard", "expiry": 14400, "timeout": 1800, "GAUID": "d6f1c3b48fe0f907e06f3e4555c2a5af1c23c56e4ee7f4a9e84f864d13d1955e"}, "error": null}
COOKIES: {'UID': 'd6f1c3b48fe0f907e06f3e4555c2a5af1c23c56e4ee7f4a9e84f864d13d1955e', 'WSSID': '"2|1:0|10:1507645862|5:WSSID|60:Q0J1QjEzdndkdGZrMmNqSVBDNmpodElTZDlpcFVSczhaY3ZRVWJuQUtPQT0=|337d65ba7d1bf41a7bb2bd1400b85ebf6995c3208bb27407e031ed98ea4686ca"'}


In [52]:
ws.send_alpha(code = '(rank((vwap-close))/rank((vwap+close)))')


simulate...
{"status": true, "result": [19086139], "error": null}

settings/all:
{"status": true, "result": {"details": {"AssetClass": {"values": [{"displayName": "Equity", "value": "EQUITY"}], "type": "enum", "paramName": "opassetclass", "displayName": "Asset Class", "dependsOn": []}, "Decay": {"type": "integer", "displayName": "Decay", "paramName": "decay"}, "Universe": {"values": {"EQUITY": {"USA": [{"displayName": "TOP3000", "value": "TOP3000"}, {"displayName": "TOP2000", "value": "TOP2000"}, {"displayName": "TOP1000", "value": "TOP1000"}, {"displayName": "TOP500", "value": "TOP500"}, {"displayName": "TOP200", "value": "TOP200"}]}}, "type": "enum", "paramName": "univid", "displayName": "Universe", "dependsOn": ["AssetClass", "Region"]}, "Region": {"values": {"EQUITY": [{"displayName": "USA", "value": "USA"}]}, "type": "enum", "paramName": "region", "displayName": "Region", "dependsOn": ["AssetClass"]}, "Delay": {"values": {"ASI": [{"displayName": "1", "value": "1"}], "USA": [{"dis

'ERROR'

In [ ]:
ws.correlation(['04fdf7f95bd64e0eb0c0e32b913ddf2a','c6d760aac3ba434c8dbc63536c8c1ab4'],'inner_corr')